### Analytics Vidhya: 52.5823376761

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)  
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline 
import seaborn as sns
import plotly.io as pio
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
data=pd.read_csv('C:/Users/saidh/Downloads/High Radius/H2HBABBA2945_G.SaiDheeraj_FoodDemand Prediction/train_GzS76OK/train.csv')
center=pd.read_csv('C:/Users/saidh/Downloads/High Radius/H2HBABBA2945_G.SaiDheeraj_FoodDemand Prediction/train_GzS76OK/fulfilment_center_info.csv')
meal=pd.read_csv('C:/Users/saidh/Downloads/High Radius/H2HBABBA2945_G.SaiDheeraj_FoodDemand Prediction/train_GzS76OK/meal_info.csv')
test=pd.read_csv('C:/Users/saidh/Downloads/High Radius/H2HBABBA2945_G.SaiDheeraj_FoodDemand Prediction/train_GzS76OK/test.csv')

##### Data Pre-Processing

In [3]:
print("The Shape of Demand dataset :",data.shape)
print("The Shape of Fulmilment Center Information dataset :",center.shape)
print("The Shape of Meal information dataset :",meal.shape)
print("The Shape of Test dataset :",test.shape)

The Shape of Demand dataset : (456548, 9)
The Shape of Fulmilment Center Information dataset : (77, 5)
The Shape of Meal information dataset : (51, 3)
The Shape of Test dataset : (32573, 8)


In [4]:
data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40


In [5]:
test['num_orders']=123456 ### Assigning random number for Target Variable of Test Data.

In [6]:
test.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1028232,146,55,1885,158.11,159.11,0,0,123456
1,1127204,146,55,1993,160.11,159.11,0,0,123456
2,1212707,146,55,2539,157.14,159.14,0,0,123456
3,1082698,146,55,2631,162.02,162.02,0,0,123456
4,1400926,146,55,1248,163.93,163.93,0,0,123456


In [7]:
center.head()

,center_id,city_code,region_code,center_type,op_area
0,11,679,56,TYPE_A,3.7
1,13,590,56,TYPE_B,6.7
2,124,590,56,TYPE_C,4.0
3,66,648,34,TYPE_A,4.1
4,94,632,34,TYPE_C,3.6


In [8]:
meal.head()

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian


In [9]:
data=pd.concat([data,test],axis=0)

In [10]:
data=data.merge(center,on='center_id',how='left')

In [11]:
data=data.merge(meal,on='meal_id',how='left')

In [12]:
data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai
1,1466964,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai
2,1346989,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai
3,1338232,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian
4,1448490,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian


In [13]:
data.isnull().sum()

id                       0
week                     0
center_id                0
meal_id                  0
checkout_price           0
base_price               0
emailer_for_promotion    0
homepage_featured        0
num_orders               0
city_code                0
region_code              0
center_type              0
op_area                  0
category                 0
cuisine                  0
dtype: int64

##### Deriving New Features

In [14]:
#Discount Amount
data['discount amount']=data['base_price']-data['checkout_price']

In [15]:
#Discount Percent
data['discount percent'] = ((data['base_price']-data['checkout_price'])/data['base_price'])*100

In [16]:
#Discount Y/N
data['discount y/n'] = [1 if x>0 else 0 for x in (data['base_price']-data['checkout_price'])]

In [17]:
data=data.sort_values(['center_id', 'meal_id', 'week']).reset_index()

In [18]:
#Compare Week Price
data['compare_week_price'] = data['checkout_price'] - data['checkout_price'].shift(1) 

In [19]:
data['compare_week_price'][data['week']==1]=0

In [20]:
data=data.sort_values(by='index').reset_index().drop(['level_0','index'],axis=1)

In [21]:
# Compare Week Price Y/N
data['compare_week_price y/n'] = [1 if x>0 else 0 for x in data['compare_week_price']]

In [22]:
data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine,discount amount,discount percent,discount y/n,compare_week_price,compare_week_price y/n
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai,15.46,10.151684,1,0.0,0
1,1466964,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai,-1.00,-0.736214,0,0.0,0
2,1346989,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai,1.00,0.736052,1,0.0,0
3,1338232,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian,98.03,22.405321,1,0.0,0
4,1448490,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian,-1.00,-0.412371,0,0.0,0


In [23]:
data.isnull().sum()

id                        0
week                      0
center_id                 0
meal_id                   0
checkout_price            0
base_price                0
emailer_for_promotion     0
homepage_featured         0
num_orders                0
city_code                 0
region_code               0
center_type               0
op_area                   0
category                  0
cuisine                   0
discount amount           0
discount percent          0
discount y/n              0
compare_week_price        0
compare_week_price y/n    0
dtype: int64

##### Train Test Split

In [24]:
train=data[data['week'].isin(range(1,146))]
test=data[data['week'].isin(range(146,156))]

In [25]:
print("The Shape of Train dataset :",train.shape)
print("The Shape of Test dataset :",test.shape)

The Shape of Train dataset : (456548, 20)
The Shape of Test dataset : (32573, 20)


Also when we checked the number of orders with respect to City, we could see that City - 590 has the highest number of orders - 18.5M orders which is almost 10M orders higher than the City with second highest number of orders - City 526 - 8.6M orders.

##### Encoding City 

As per our observation from our barchart of the City against the number of orders. There the high significant difference between the Top 3 cities which has the highest number of orders. Therefore, in our first approach we will encode the City with Highest No. of Orders as CH1, City with 2nd Highest No. of Orders as CH2 and City with 3rd Highest No. of Orders as CH3 and rest all of the cities which does not have much significant differences between the number of orders as CH4.

In [26]:
city4={590:'CH1', 526:'CH2', 638:'CH3'}
data['city_enc_4']=data['city_code'].map(city4)
data['city_enc_4']=data['city_enc_4'].fillna('CH4')

In [27]:
data['city_enc_4'].value_counts()

CH4    362341
CH1     58708
CH2     46589
CH3     21483
Name: city_enc_4, dtype: int64

In [28]:
data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine,discount amount,discount percent,discount y/n,compare_week_price,compare_week_price y/n,city_enc_4
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai,15.46,10.151684,1,0.0,0,CH4
1,1466964,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai,-1.00,-0.736214,0,0.0,0,CH4
2,1346989,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai,1.00,0.736052,1,0.0,0,CH4
3,1338232,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian,98.03,22.405321,1,0.0,0,CH4
4,1448490,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian,-1.00,-0.412371,0,0.0,0,CH4


In [29]:
data.isnull().sum()

id                        0
week                      0
center_id                 0
meal_id                   0
checkout_price            0
base_price                0
emailer_for_promotion     0
homepage_featured         0
num_orders                0
city_code                 0
region_code               0
center_type               0
op_area                   0
category                  0
cuisine                   0
discount amount           0
discount percent          0
discount y/n              0
compare_week_price        0
compare_week_price y/n    0
city_enc_4                0
dtype: int64

##### Copying to New DataFrame

In [30]:
datax=data.copy()
datax.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,region_code,center_type,op_area,category,cuisine,discount amount,discount percent,discount y/n,compare_week_price,compare_week_price y/n,city_enc_4
0,1379560,1,55,1885,136.83,152.29,0,0,177,647,56,TYPE_C,2.0,Beverages,Thai,15.46,10.151684,1,0.0,0,CH4
1,1466964,1,55,1993,136.83,135.83,0,0,270,647,56,TYPE_C,2.0,Beverages,Thai,-1.00,-0.736214,0,0.0,0,CH4
2,1346989,1,55,2539,134.86,135.86,0,0,189,647,56,TYPE_C,2.0,Beverages,Thai,1.00,0.736052,1,0.0,0,CH4
3,1338232,1,55,2139,339.50,437.53,0,0,54,647,56,TYPE_C,2.0,Beverages,Indian,98.03,22.405321,1,0.0,0,CH4
4,1448490,1,55,2631,243.50,242.50,0,0,40,647,56,TYPE_C,2.0,Beverages,Indian,-1.00,-0.412371,0,0.0,0,CH4


##### Enoding all the Categorical Features

In [31]:
datax['center_id']=datax['center_id'].astype('object')
datax['meal_id']=datax['meal_id'].astype('object')
datax['region_code']=datax['region_code'].astype('object')

In [32]:
obj=datax[['center_id','meal_id','region_code','center_type','category','cuisine','city_enc_4']]
num=datax.drop(['center_id','meal_id','region_code','center_type','category','cuisine','city_enc_4'],axis=1)

In [33]:
encode1=pd.get_dummies(obj,drop_first = True)

In [34]:
datax=pd.concat([num,encode1],axis=1)

In [35]:
datax.head()

,id,week,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,city_code,op_area,discount amount,discount percent,discount y/n,compare_week_price,compare_week_price y/n,center_id_11,center_id_13,center_id_14,center_id_17,center_id_20,center_id_23,center_id_24,center_id_26,center_id_27,center_id_29,center_id_30,center_id_32,center_id_34,center_id_36,center_id_39,center_id_41,center_id_42,center_id_43,center_id_50,center_id_51,center_id_52,center_id_53,center_id_55,center_id_57,center_id_58,center_id_59,center_id_61,center_id_64,center_id_65,center_id_66,center_id_67,center_id_68,center_id_72,center_id_73,center_id_74,center_id_75,center_id_76,center_id_77,center_id_80,center_id_81,center_id_83,center_id_86,center_id_88,center_id_89,center_id_91,center_id_92,center_id_93,center_id_94,center_id_97,center_id_99,center_id_101,center_id_102,center_id_104,center_id_106,center_id_108,center_id_109,center_id_110,center_id_113,center_id_124,center_id_126,center_id_129,center_id_132,center_id_137,center_id_139,center_id_143,center_id_145,center_id_146,center_id_149,center_id_152,center_id_153,center_id_157,center_id_161,center_id_162,center_id_174,center_id_177,center_id_186,meal_id_1109,meal_id_1198,meal_id_1207,meal_id_1216,meal_id_1230,meal_id_1247,meal_id_1248,meal_id_1311,meal_id_1438,meal_id_1445,meal_id_1525,meal_id_1543,meal_id_1558,meal_id_1571,meal_id_1727,meal_id_1754,meal_id_1770,meal_id_1778,meal_id_1803,meal_id_1847,meal_id_1878,meal_id_1885,meal_id_1902,meal_id_1962,meal_id_1971,meal_id_1993,meal_id_2104,meal_id_2126,meal_id_2139,meal_id_2290,meal_id_2304,meal_id_2306,meal_id_2322,meal_id_2444,meal_id_2490,meal_id_2492,meal_id_2494,meal_id_2539,meal_id_2569,meal_id_2577,meal_id_2581,meal_id_2631,meal_id_2640,meal_id_2664,meal_id_2704,meal_id_2707,meal_id_2760,meal_id_2826,meal_id_2867,meal_id_2956,region_code_34,region_code_35,region_code_56,region_code_71,region_code_77,region_code_85,region_code_93,center_type_TYPE_B,center_type_TYPE_C,category_Biryani,category_Desert,category_Extras,category_Fish,category_Other Snacks,category_Pasta,category_Pizza,category_Rice Bowl,category_Salad,category_Sandwich,category_Seafood,category_Soup,category_Starters,cuisine_Indian,cuisine_Italian,cuisine_Thai,city_enc_4_CH2,city_enc_4_CH3,city_enc_4_CH4
0,1379560,1,136.83,152.29,0,0,177,647,2.0,15.46,10.151684,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,1466964,1,136.83,135.83,0,0,270,647,2.0,-1.00,-0.736214,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
2,1346989,1,134.86,135.86,0,0,189,647,2.0,1.00,0.736052,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1338232,1,339.50,437.53,0,0,54,647,2.0,98.03,22.405321,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4,1448490,1,243.50,242.50,0,0,40,647,2.0,-1.00,-0.412371,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [36]:
abs(train.corr()['num_orders']).sort_values(ascending=False)

num_orders                1.000000
homepage_featured         0.294490
checkout_price            0.282108
emailer_for_promotion     0.277147
base_price                0.222306
discount percent          0.205188
op_area                   0.176976
discount amount           0.152950
compare_week_price        0.137849
discount y/n              0.111766
center_id                 0.053035
city_code                 0.041596
region_code               0.029744
compare_week_price y/n    0.022898
week                      0.017210
meal_id                   0.010597
id                        0.000702
Name: num_orders, dtype: float64

##### Base Model

In [37]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [38]:
sc=StandardScaler()

cat=datax.drop(['checkout_price','base_price','discount amount','discount percent','compare_week_price'],axis=1)
num=datax[['checkout_price','base_price','discount amount','discount percent','compare_week_price']]
scal= pd.DataFrame(sc.fit_transform(num),columns=num.columns)
datas=pd.concat([scal,cat],axis=1)

In [39]:
datay=datas.copy()

In [40]:
datay['Quarter']=(datas['week']/13).astype('int64')

In [41]:
datay['Quarter'] = datay['Quarter'].map({0:'Q1',
                         1:'Q2',
                         2:'Q3',
                         3:'Q4',
                         4:'Q1',
                         5:'Q2',
                         6:'Q3',
                         7:'Q4',
                         8:'Q1',
                         9:'Q2',
                         10:'Q3',
                         11:'Q4'})

In [42]:
datay['Quarter'].value_counts()

Q4    124839
Q3    124225
Q2    122386
Q1    117671
Name: Quarter, dtype: int64

In [43]:
datay['Year']=(datas['week']/52).astype('int64')

In [44]:
datay['Year'] = datay['Year'].map({0:'Y1',
                         1:'Y2',
                         2:'Y3'})

In [45]:
objy=datay[['Quarter', 'Year']]
numy=datay.drop(['Quarter', 'Year'],axis=1)

encode1y=pd.get_dummies(objy,drop_first = True)
encode1y.head()

datay=pd.concat([numy,encode1y],axis=1)

In [46]:
datay.head()

,checkout_price,base_price,discount amount,discount percent,compare_week_price,id,week,emailer_for_promotion,homepage_featured,num_orders,city_code,op_area,discount y/n,compare_week_price y/n,center_id_11,center_id_13,center_id_14,center_id_17,center_id_20,center_id_23,center_id_24,center_id_26,center_id_27,center_id_29,center_id_30,center_id_32,center_id_34,center_id_36,center_id_39,center_id_41,center_id_42,center_id_43,center_id_50,center_id_51,center_id_52,center_id_53,center_id_55,center_id_57,center_id_58,center_id_59,center_id_61,center_id_64,center_id_65,center_id_66,center_id_67,center_id_68,center_id_72,center_id_73,center_id_74,center_id_75,center_id_76,center_id_77,center_id_80,center_id_81,center_id_83,center_id_86,center_id_88,center_id_89,center_id_91,center_id_92,center_id_93,center_id_94,center_id_97,center_id_99,center_id_101,center_id_102,center_id_104,center_id_106,center_id_108,center_id_109,center_id_110,center_id_113,center_id_124,center_id_126,center_id_129,center_id_132,center_id_137,center_id_139,center_id_143,center_id_145,center_id_146,center_id_149,center_id_152,center_id_153,center_id_157,center_id_161,center_id_162,center_id_174,center_id_177,center_id_186,meal_id_1109,meal_id_1198,meal_id_1207,meal_id_1216,meal_id_1230,meal_id_1247,meal_id_1248,meal_id_1311,meal_id_1438,meal_id_1445,meal_id_1525,meal_id_1543,meal_id_1558,meal_id_1571,meal_id_1727,meal_id_1754,meal_id_1770,meal_id_1778,meal_id_1803,meal_id_1847,meal_id_1878,meal_id_1885,meal_id_1902,meal_id_1962,meal_id_1971,meal_id_1993,meal_id_2104,meal_id_2126,meal_id_2139,meal_id_2290,meal_id_2304,meal_id_2306,meal_id_2322,meal_id_2444,meal_id_2490,meal_id_2492,meal_id_2494,meal_id_2539,meal_id_2569,meal_id_2577,meal_id_2581,meal_id_2631,meal_id_2640,meal_id_2664,meal_id_2704,meal_id_2707,meal_id_2760,meal_id_2826,meal_id_2867,meal_id_2956,region_code_34,region_code_35,region_code_56,region_code_71,region_code_77,region_code_85,region_code_93,center_type_TYPE_B,center_type_TYPE_C,category_Biryani,category_Desert,category_Extras,category_Fish,category_Other Snacks,category_Pasta,category_Pizza,category_Rice Bowl,category_Salad,category_Sandwich,category_Seafood,category_Soup,category_Starters,cuisine_Indian,cuisine_Italian,cuisine_Thai,city_enc_4_CH2,city_enc_4_CH3,city_enc_4_CH4,Quarter_Q2,Quarter_Q3,Quarter_Q4,Year_Y2,Year_Y3
0,-1.281184,-1.259867,-0.124498,0.399910,-0.003326,1379560,1,0,0,177,647,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,-1.281184,-1.362517,-0.467581,-0.562824,-0.003326,1466964,1,0,0,270,647,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
2,-1.294058,-1.362329,-0.425894,-0.432643,-0.003326,1346989,1,0,0,189,647,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,0.043267,0.518969,1.596548,1.483406,-0.003326,1338232,1,0,0,54,647,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,-0.584095,-0.697293,-0.467581,-0.534189,-0.003326,1

##### Applying Log Transformation on the Target Feature

In [47]:
datay['num_orders']=np.log1p(datay['num_orders'])

In [48]:
train=datay[datay['week'].isin(range(1,146))]
def outliers_3(col):
    q3=round(train[col].quantile(0.75),6)
    q1=round(train[col].quantile(0.25),6)
    iqr=q3-q1
    lw = q1 - (3*iqr)
    hw = q3 + (3*iqr)  
    uo=train[train[col]>hw].shape[0]
    lo=train[train[col]<lw].shape[0]
    print('Number of Upper Outliers :',uo)
    print('Number of Lower Outliers :',lo)
    print('Percentage of Outliers :',((uo+lo)/train.shape[0])*100)

In [49]:
outliers_3('num_orders')

Number of Upper Outliers : 0
Number of Lower Outliers : 0
Percentage of Outliers : 0.0


In [50]:
datay.head()

,checkout_price,base_price,discount amount,discount percent,compare_week_price,id,week,emailer_for_promotion,homepage_featured,num_orders,city_code,op_area,discount y/n,compare_week_price y/n,center_id_11,center_id_13,center_id_14,center_id_17,center_id_20,center_id_23,center_id_24,center_id_26,center_id_27,center_id_29,center_id_30,center_id_32,center_id_34,center_id_36,center_id_39,center_id_41,center_id_42,center_id_43,center_id_50,center_id_51,center_id_52,center_id_53,center_id_55,center_id_57,center_id_58,center_id_59,center_id_61,center_id_64,center_id_65,center_id_66,center_id_67,center_id_68,center_id_72,center_id_73,center_id_74,center_id_75,center_id_76,center_id_77,center_id_80,center_id_81,center_id_83,center_id_86,center_id_88,center_id_89,center_id_91,center_id_92,center_id_93,center_id_94,center_id_97,center_id_99,center_id_101,center_id_102,center_id_104,center_id_106,center_id_108,center_id_109,center_id_110,center_id_113,center_id_124,center_id_126,center_id_129,center_id_132,center_id_137,center_id_139,center_id_143,center_id_145,center_id_146,center_id_149,center_id_152,center_id_153,center_id_157,center_id_161,center_id_162,center_id_174,center_id_177,center_id_186,meal_id_1109,meal_id_1198,meal_id_1207,meal_id_1216,meal_id_1230,meal_id_1247,meal_id_1248,meal_id_1311,meal_id_1438,meal_id_1445,meal_id_1525,meal_id_1543,meal_id_1558,meal_id_1571,meal_id_1727,meal_id_1754,meal_id_1770,meal_id_1778,meal_id_1803,meal_id_1847,meal_id_1878,meal_id_1885,meal_id_1902,meal_id_1962,meal_id_1971,meal_id_1993,meal_id_2104,meal_id_2126,meal_id_2139,meal_id_2290,meal_id_2304,meal_id_2306,meal_id_2322,meal_id_2444,meal_id_2490,meal_id_2492,meal_id_2494,meal_id_2539,meal_id_2569,meal_id_2577,meal_id_2581,meal_id_2631,meal_id_2640,meal_id_2664,meal_id_2704,meal_id_2707,meal_id_2760,meal_id_2826,meal_id_2867,meal_id_2956,region_code_34,region_code_35,region_code_56,region_code_71,region_code_77,region_code_85,region_code_93,center_type_TYPE_B,center_type_TYPE_C,category_Biryani,category_Desert,category_Extras,category_Fish,category_Other Snacks,category_Pasta,category_Pizza,category_Rice Bowl,category_Salad,category_Sandwich,category_Seafood,category_Soup,category_Starters,cuisine_Indian,cuisine_Italian,cuisine_Thai,city_enc_4_CH2,city_enc_4_CH3,city_enc_4_CH4,Quarter_Q2,Quarter_Q3,Quarter_Q4,Year_Y2,Year_Y3
0,-1.281184,-1.259867,-0.124498,0.399910,-0.003326,1379560,1,0,0,5.181784,647,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,-1.281184,-1.362517,-0.467581,-0.562824,-0.003326,1466964,1,0,0,5.602119,647,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
2,-1.294058,-1.362329,-0.425894,-0.432643,-0.003326,1346989,1,0,0,5.247024,647,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,0.043267,0.518969,1.596548,1.483406,-0.003326,1338232,1,0,0,4.007333,647,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,-0.584095,-0.697293,-0.467581,

In [58]:
train=datay[datay['week'].isin(range(1,146))]
test=datay[datay['week'].isin(range(146,156))]

In [59]:
X_train=train.drop(['id','num_orders','week','discount amount','city_code','Quarter_Q2','Quarter_Q3','base_price','discount percent'],axis=1)
y_train=train['num_orders']

In [61]:
X_test=test.drop(['id','num_orders','week','discount amount','city_code','Quarter_Q2','Quarter_Q3','base_price','discount percent'],axis=1)

In [51]:
from catboost import CatBoostRegressor

In [53]:
model = CatBoostRegressor()

In [72]:
params = {'learning_rate': [0.03, 0.05, 0.07, 0.1], 
          #'min_child_weight':[4,5], 
          #'gamma':[i/10.0 for i in range(3,6)],  
          #'subsample':[i/10.0 for i in range(6,11)],
          #'colsample_bytree':[i/10.0 for i in range(6,11)], 
          'max_depth': [2,3,4,5, 6, 7,8],
          'n_estimators': [200, 500]}

In [73]:
from sklearn.model_selection import GridSearchCV

In [74]:
catb_grid = GridSearchCV(model, params, cv = 2, n_jobs = 5, verbose=True)

In [75]:
catb_grid.fit(X_train, y_train)

Fitting 2 folds for each of 56 candidates, totalling 112 fits
0:	learn: 1.1350813	total: 294ms	remaining: 2m 26s
1:	learn: 1.0766885	total: 440ms	remaining: 1m 49s
2:	learn: 1.0265020	total: 680ms	remaining: 1m 52s
3:	learn: 0.9839391	total: 861ms	remaining: 1m 46s
4:	learn: 0.9469432	total: 1.03s	remaining: 1m 42s
5:	learn: 0.9149917	total: 1.25s	remaining: 1m 43s
6:	learn: 0.8880317	total: 1.39s	remaining: 1m 38s
7:	learn: 0.8637600	total: 1.57s	remaining: 1m 36s
8:	learn: 0.8413095	total: 1.75s	remaining: 1m 35s
9:	learn: 0.8234844	total: 1.91s	remaining: 1m 33s
10:	learn: 0.8069022	total: 2.05s	remaining: 1m 31s
11:	learn: 0.7929457	total: 2.22s	remaining: 1m 30s
12:	learn: 0.7792207	total: 2.34s	remaining: 1m 27s
13:	learn: 0.7679512	total: 2.48s	remaining: 1m 26s
14:	learn: 0.7577524	total: 2.63s	remaining: 1m 24s
15:	learn: 0.7465474	total: 2.79s	remaining: 1m 24s
16:	learn: 0.7372703	total: 2.97s	remaining: 1m 24s
17:	learn: 0.7292517	total: 3.13s	remaining: 1m 23s
18:	learn: 0

159:	learn: 0.5525173	total: 23.8s	remaining: 50.5s
160:	learn: 0.5522673	total: 23.9s	remaining: 50.3s
161:	learn: 0.5519834	total: 24.1s	remaining: 50.2s
162:	learn: 0.5515863	total: 24.2s	remaining: 50.1s
163:	learn: 0.5511866	total: 24.4s	remaining: 49.9s
164:	learn: 0.5509103	total: 24.5s	remaining: 49.7s
165:	learn: 0.5506642	total: 24.6s	remaining: 49.6s
166:	learn: 0.5502855	total: 24.8s	remaining: 49.4s
167:	learn: 0.5500327	total: 24.9s	remaining: 49.2s
168:	learn: 0.5497031	total: 25s	remaining: 49s
169:	learn: 0.5493397	total: 25.2s	remaining: 48.9s
170:	learn: 0.5490154	total: 25.3s	remaining: 48.7s
171:	learn: 0.5487447	total: 25.4s	remaining: 48.5s
172:	learn: 0.5484532	total: 25.6s	remaining: 48.3s
173:	learn: 0.5481550	total: 25.7s	remaining: 48.2s
174:	learn: 0.5477571	total: 25.8s	remaining: 48s
175:	learn: 0.5474526	total: 26s	remaining: 47.8s
176:	learn: 0.5471443	total: 26.1s	remaining: 47.6s
177:	learn: 0.5467369	total: 26.2s	remaining: 47.5s
178:	learn: 0.546420

319:	learn: 0.5171010	total: 46.4s	remaining: 26.1s
320:	learn: 0.5169632	total: 46.5s	remaining: 26s
321:	learn: 0.5168634	total: 46.7s	remaining: 25.8s
322:	learn: 0.5167126	total: 46.8s	remaining: 25.7s
323:	learn: 0.5166021	total: 47s	remaining: 25.5s
324:	learn: 0.5164737	total: 47.1s	remaining: 25.4s
325:	learn: 0.5163564	total: 47.3s	remaining: 25.2s
326:	learn: 0.5161886	total: 47.5s	remaining: 25.1s
327:	learn: 0.5160722	total: 47.6s	remaining: 25s
328:	learn: 0.5159772	total: 47.8s	remaining: 24.8s
329:	learn: 0.5158624	total: 47.9s	remaining: 24.7s
330:	learn: 0.5157201	total: 48.1s	remaining: 24.5s
331:	learn: 0.5155869	total: 48.2s	remaining: 24.4s
332:	learn: 0.5154647	total: 48.4s	remaining: 24.3s
333:	learn: 0.5153434	total: 48.6s	remaining: 24.2s
334:	learn: 0.5151743	total: 48.9s	remaining: 24.1s
335:	learn: 0.5150647	total: 49s	remaining: 23.9s
336:	learn: 0.5149016	total: 49.2s	remaining: 23.8s
337:	learn: 0.5147604	total: 49.3s	remaining: 23.6s
338:	learn: 0.514635

479:	learn: 0.4998710	total: 1m 9s	remaining: 2.89s
480:	learn: 0.4997794	total: 1m 9s	remaining: 2.75s
481:	learn: 0.4997255	total: 1m 9s	remaining: 2.6s
482:	learn: 0.4996415	total: 1m 9s	remaining: 2.46s
483:	learn: 0.4995721	total: 1m 9s	remaining: 2.31s
484:	learn: 0.4994926	total: 1m 10s	remaining: 2.17s
485:	learn: 0.4993834	total: 1m 10s	remaining: 2.02s
486:	learn: 0.4992973	total: 1m 10s	remaining: 1.88s
487:	learn: 0.4992297	total: 1m 10s	remaining: 1.74s
488:	learn: 0.4991831	total: 1m 10s	remaining: 1.59s
489:	learn: 0.4990932	total: 1m 10s	remaining: 1.45s
490:	learn: 0.4990227	total: 1m 11s	remaining: 1.3s
491:	learn: 0.4989586	total: 1m 11s	remaining: 1.16s
492:	learn: 0.4988585	total: 1m 11s	remaining: 1.01s
493:	learn: 0.4987831	total: 1m 11s	remaining: 868ms
494:	learn: 0.4987179	total: 1m 11s	remaining: 723ms
495:	learn: 0.4986039	total: 1m 11s	remaining: 578ms
496:	learn: 0.4985288	total: 1m 11s	remaining: 434ms
497:	learn: 0.4984419	total: 1m 11s	remaining: 289ms


GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostRegressor object at 0x000001E914739DA0>,
             n_jobs=5,
             param_grid={'learning_rate': [0.03, 0.05, 0.07, 0.1],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8],
                         'n_estimators': [200, 500]},
             verbose=True)

In [76]:
catb_grid.best_params_

{'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 500}

In [80]:
catb = CatBoostRegressor(learning_rate = 0.1, max_depth= 8, n_estimators= 500)

In [81]:
catb.fit(X_train, y_train)

0:	learn: 1.1350813	total: 263ms	remaining: 2m 11s
1:	learn: 1.0766885	total: 454ms	remaining: 1m 52s
2:	learn: 1.0265020	total: 619ms	remaining: 1m 42s
3:	learn: 0.9839391	total: 799ms	remaining: 1m 39s
4:	learn: 0.9469432	total: 982ms	remaining: 1m 37s
5:	learn: 0.9149917	total: 1.19s	remaining: 1m 37s
6:	learn: 0.8880317	total: 1.38s	remaining: 1m 37s
7:	learn: 0.8637600	total: 1.66s	remaining: 1m 42s
8:	learn: 0.8413095	total: 1.84s	remaining: 1m 40s
9:	learn: 0.8234844	total: 2.01s	remaining: 1m 38s
10:	learn: 0.8069022	total: 2.17s	remaining: 1m 36s
11:	learn: 0.7929457	total: 2.34s	remaining: 1m 35s
12:	learn: 0.7792207	total: 2.5s	remaining: 1m 33s
13:	learn: 0.7679512	total: 2.65s	remaining: 1m 31s
14:	learn: 0.7577524	total: 2.81s	remaining: 1m 30s
15:	learn: 0.7465474	total: 2.98s	remaining: 1m 30s
16:	learn: 0.7372703	total: 3.15s	remaining: 1m 29s
17:	learn: 0.7292517	total: 3.35s	remaining: 1m 29s
18:	learn: 0.7222947	total: 3.55s	remaining: 1m 29s
19:	learn: 0.7160194	to

160:	learn: 0.5522673	total: 26.8s	remaining: 56.5s
161:	learn: 0.5519834	total: 27s	remaining: 56.2s
162:	learn: 0.5515863	total: 27.1s	remaining: 56s
163:	learn: 0.5511866	total: 27.3s	remaining: 55.9s
164:	learn: 0.5509103	total: 27.4s	remaining: 55.7s
165:	learn: 0.5506642	total: 27.6s	remaining: 55.5s
166:	learn: 0.5502855	total: 27.8s	remaining: 55.4s
167:	learn: 0.5500327	total: 28s	remaining: 55.2s
168:	learn: 0.5497031	total: 28.1s	remaining: 55.1s
169:	learn: 0.5493397	total: 28.3s	remaining: 55s
170:	learn: 0.5490154	total: 28.5s	remaining: 54.9s
171:	learn: 0.5487447	total: 28.7s	remaining: 54.7s
172:	learn: 0.5484532	total: 28.8s	remaining: 54.5s
173:	learn: 0.5481550	total: 29s	remaining: 54.4s
174:	learn: 0.5477571	total: 29.2s	remaining: 54.2s
175:	learn: 0.5474526	total: 29.3s	remaining: 54s
176:	learn: 0.5471443	total: 29.5s	remaining: 53.8s
177:	learn: 0.5467369	total: 29.6s	remaining: 53.6s
178:	learn: 0.5464201	total: 29.8s	remaining: 53.5s
179:	learn: 0.5461418	to

320:	learn: 0.5169632	total: 55.3s	remaining: 30.9s
321:	learn: 0.5168634	total: 55.5s	remaining: 30.7s
322:	learn: 0.5167126	total: 55.7s	remaining: 30.5s
323:	learn: 0.5166021	total: 55.9s	remaining: 30.4s
324:	learn: 0.5164737	total: 56.1s	remaining: 30.2s
325:	learn: 0.5163564	total: 56.3s	remaining: 30s
326:	learn: 0.5161886	total: 56.5s	remaining: 29.9s
327:	learn: 0.5160722	total: 56.6s	remaining: 29.7s
328:	learn: 0.5159772	total: 56.8s	remaining: 29.5s
329:	learn: 0.5158624	total: 57s	remaining: 29.3s
330:	learn: 0.5157201	total: 57.1s	remaining: 29.2s
331:	learn: 0.5155869	total: 57.3s	remaining: 29s
332:	learn: 0.5154647	total: 57.5s	remaining: 28.8s
333:	learn: 0.5153434	total: 57.6s	remaining: 28.6s
334:	learn: 0.5151743	total: 57.8s	remaining: 28.5s
335:	learn: 0.5150647	total: 58s	remaining: 28.3s
336:	learn: 0.5149016	total: 58.1s	remaining: 28.1s
337:	learn: 0.5147604	total: 58.3s	remaining: 27.9s
338:	learn: 0.5146352	total: 58.5s	remaining: 27.8s
339:	learn: 0.514510

478:	learn: 0.4999627	total: 1m 22s	remaining: 3.64s
479:	learn: 0.4998710	total: 1m 23s	remaining: 3.46s
480:	learn: 0.4997794	total: 1m 23s	remaining: 3.29s
481:	learn: 0.4997255	total: 1m 23s	remaining: 3.12s
482:	learn: 0.4996415	total: 1m 23s	remaining: 2.94s
483:	learn: 0.4995721	total: 1m 23s	remaining: 2.77s
484:	learn: 0.4994926	total: 1m 23s	remaining: 2.6s
485:	learn: 0.4993834	total: 1m 24s	remaining: 2.42s
486:	learn: 0.4992973	total: 1m 24s	remaining: 2.25s
487:	learn: 0.4992297	total: 1m 24s	remaining: 2.08s
488:	learn: 0.4991831	total: 1m 24s	remaining: 1.9s
489:	learn: 0.4990932	total: 1m 24s	remaining: 1.73s
490:	learn: 0.4990227	total: 1m 24s	remaining: 1.56s
491:	learn: 0.4989586	total: 1m 25s	remaining: 1.38s
492:	learn: 0.4988585	total: 1m 25s	remaining: 1.21s
493:	learn: 0.4987831	total: 1m 25s	remaining: 1.04s
494:	learn: 0.4987179	total: 1m 25s	remaining: 864ms
495:	learn: 0.4986039	total: 1m 25s	remaining: 692ms
496:	learn: 0.4985288	total: 1m 25s	remaining: 5

In [86]:
Catpred=catb.predict(X_test)

In [87]:
Result = pd.DataFrame(Catpred)
Result=np.expm1(Result).astype('int64')
Submission = pd.DataFrame(columns=['id', 'num_orders'])
Submission['id'] = test['id']
Submission['num_orders'] = Result.values
Submission.to_csv('Trail6_Submission.csv', index=False)

### Analytics Vidhya: 53.8

##### Model 2

In [64]:
train=datay[datay['week'].isin(range(1,146))]
test=datay[datay['week'].isin(range(146,156))]


X_train=train.drop(['id','num_orders','week','discount amount','city_code','Quarter_Q2','Quarter_Q3','base_price','discount percent'],axis=1)
y_train=train['num_orders']

X_test=test.drop(['id','num_orders','week','discount amount','city_code','Quarter_Q2','Quarter_Q3','base_price','discount percent'],axis=1)

CGB=CatBoostRegressor(learning_rate=0.1,loss_function='RMSE',max_depth=8,verbose=False)

CGB.fit(X_train, y_train)
CGBpred=CGB.predict(X_test)

In [65]:
Result=pd.DataFrame(CGBpred)
Result=np.expm1(Result).astype('int64')
Submission1 = pd.DataFrame(columns=['id', 'num_orders'])
Submission1['id'] = test['id']
Submission1['num_orders'] = Result.values
Submission1.to_csv('Submission_CatBoost.csv', index=False)

### Analytics Vidhya: 52.5823376761

In [66]:
Submission.head()

,id,num_orders
456548,1028232,177
456549,1127204,99
456550,1212707,141
456551,1082698,55
456552,1400926,35


In [67]:
Submission1.head()

,id,num_orders
456548,1028232,191
456549,1127204,145
456550,1212707,115
456551,1082698,45
456552,1400926,29
